In [1]:
import os


In [2]:
os.chdir("d://Data_Analysis//End-to-End_Text_Summarization//Text-Summarizer//src//")
%pwd

'd:\\Data_Analysis\\End-to-End_Text_Summarization\\Text-Summarizer\\src'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [5]:
os.chdir("d://Data_Analysis//End-to-End_Text_Summarization//Text-Summarizer//")

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

d:\Data_Analysis\End-to-End_Text_Summarization\Text-Summarizer\textsummarizer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-06 19:58:05,152: INFO: config: PyTorch version 2.5.1 available.]


In [8]:
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config


    
#     def train(self):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
#         #loading data 
#         dataset_samsum_pt = load_from_disk(self.config.data_path)

#         # trainer_args = TrainingArguments(
#         #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
#         #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
#         #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
#         #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
#         #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
#         # ) 


#         trainer_args = TrainingArguments(
#             output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
#             per_device_train_batch_size=1, per_device_eval_batch_size=1,
#             weight_decay=0.01, logging_steps=10,
#             evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
#             gradient_accumulation_steps=16
#         ) 

#         trainer = Trainer(model=model_pegasus, args=trainer_args,
#                   tokenizer=tokenizer, data_collator=seq2seq_data_collator,
#                   train_dataset=dataset_samsum_pt["train"], 
#                   eval_dataset=dataset_samsum_pt["validation"])
        
#         trainer.train()

#         ## Save model
#         model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
#         ## Save tokenizer
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def preprocess_data(self, examples):
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        # inputs = examples["dialogue"]
        # targets = examples["summary"]

        inputs = str(examples["dialogue"]) if not isinstance(examples["dialogue"], str) else examples["dialogue"]
        targets = str(examples["summary"]) if not isinstance(examples["summary"], str) else examples["summary"]


         # Tokenize the dialogue column as input_ids
        model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True, return_tensors="pt")

        # Tokenize the summary column as labels
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=128, truncation=True, padding=True, return_tensors="pt", text_target=True)

        model_inputs["labels"] = labels["input_ids"]
        
        return model_inputs
    
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum_pt.map(self.preprocess_data, batched=True)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            warmup_steps=self.config.warmup_steps,
            per_device_eval_batch_size= self.config.per_device_train_batch_size, 
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps=1e6,
            # save_steps=int(self.config.save_steps),
            gradient_accumulation_steps=self.config.gradient_accumulation_steps, 
            remove_unused_columns=False
        )

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
        #     per_device_train_batch_size=1, per_device_eval_batch_size=1,
        #     weight_decay=0.01, logging_steps=10,
        #     evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
        #     gradient_accumulation_steps=16
        # ) 

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        #save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e



[2024-11-06 19:58:08,291: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2024-11-06 19:58:08,291: INFO: common: yaml_file: params.yaml loaded successfully]
[2024-11-06 19:58:08,291: INFO: common: created directory at : artifacts]
[2024-11-06 19:58:08,300: INFO: common: created directory at : artifacts/model_trainer]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Data_Analysis\End-to-End_Text_Summarization\Text-Summarizer\textsummarizer\Lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map:   0%|          | 0/14732 [00:01<?, ? examples/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).